NB! Повторно код я запускал только начиная с раздела 1.3, так как в 1.1 я написал краулер который довольно долго собирает данные (и в какой-то момент ломается), а в 1.2 я записываю результат его работы в json файл, который потом при перезапуске считываю в 1.3

Для краулера используется chromedrive, поэтому он не запустится, если у вас на компьютере его нет.

# 1. Получаем данные

Мне хотелось найти сайт, где будет много отрицательных отзывов, и у меня получилось! На https://otzovik.com/reviews/internet-sayt_www_avito_ru/ очень не любят Авито, поэтому негативных отзывов я там нашел намного больше, чем положительных

## 1.1. Краулер для сайта https://otzovik.com с отзывами на Авито

Краулер на основе селениума

In [1]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
driver = webdriver.Chrome()

In [3]:
def parse_review(review): #парсинг отдельного отзыва
    global positive #списки положительных и отрицательных отзывов соответственно
    global negative
    url_rev = 'https://otzovik.com' + review.attrs['href']  #ссылка на отзыв
    req = driver.get(url_rev)
    page = driver.page_source
    if len(page) > 0:
        soup = BeautifulSoup(page, 'html.parser')
        full_text = soup.find('div', {'class': 'review-body description'}).text #текст отзыва
        rating = soup.find('div', {'class': 'product-rating tooltip-right'}).attrs['title'][-1] #оценка
        if int(rating) < 3: #если оценка выше 3, то отзыв считается положительным, если ниже -- отрицательным
            negative.append(full_text)
        if int(rating) > 3:
            positive.append(full_text)

In [4]:
def parse_page(page_n): #парсинг страницы с отзывами
    time.sleep(2)
    url = f'https://otzovik.com/reviews/internet-sayt_www_avito_ru/{page_n}/'
    req = driver.get(url)
    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')
    reviews = soup.find_all('a', {'class': 'review-btn review-read-link'}) #нахожу отдельные отзывы на странице
    for review in reviews:
        time.sleep(2)
        parse_review(review) #отправляю отзыв на парсинг

In [9]:
positive = []
negative = []
for i in range(2,50): #решаю распарсить страницы от 2 до 50
    parse_page(i)

AttributeError: 'NoneType' object has no attribute 'text'

Мне закрыли доступ к сайту до того, как он распарсил 50 страниц(

In [10]:
len(positive)

95

In [11]:
len(negative)

739

Положительных отзывов намного меньше отрицательных, поэтому от отрицательных может быть полезно избавиться

## 1.2. Записываю резулььтат в json

In [19]:
import json

In [15]:
dct = {}
dct['positive'] = positive
dct['negative'] = negative

In [24]:
with open('reviews.json', 'w', encoding='utf-8') as f:
    json.dump(dct, f, ensure_ascii=False, indent='\t')

## 1.3. Вынимаю данные из json

In [2]:
import json

In [3]:
with open('reviews.json', 'r', encoding='utf-8') as f:
    reviews = json.load(f)

# 2. Препроцессинг

## 2.0. Чищу отзывы

Избавлюсь от слишком большого количества отрицательных отзывов

In [4]:
import numpy as np
import random

In [5]:
np.mean([len(i) for i in reviews['positive']])

1704.0526315789473

In [6]:
len_neg = np.array([len(i) for i in reviews['negative']])

In [7]:
np.mean(len_neg[len_neg > 535])

1703.6056572379368

In [8]:
np.count_nonzero(len_neg[len_neg > 535])

601

Решаю оставить 100 случайных отзывов длиннее 535 символов

In [9]:
random.seed(42)
reviews['negative'] = random.choices(np.array(reviews['negative'])[len_neg > 535], k=100)

Формирую тестовую выборку из 5 отрицательных и 5 положительных отзывов

In [10]:
test = reviews['negative'][-5:]
reviews['negative'] = reviews['negative'][:-5]

In [11]:
test.extend(reviews['positive'][-5:])
reviews['positive'] = reviews['positive'][:-5]

In [12]:
y = [-1,-1,-1,-1,-1,1,1,1,1,1]

## 2.1. Токенизация и нижний регистр

In [13]:
from nltk.tokenize import casual_tokenize

In [14]:
neg_prep = []
pos_prep = []
for review in reviews['negative']:
    neg_prep.append(casual_tokenize(review, preserve_case=False))
for review in reviews['positive']:
    pos_prep.append(casual_tokenize(review, preserve_case=False))

## 2.2. Начальная форма

In [15]:
from pymorphy3 import MorphAnalyzer

In [16]:
morph = MorphAnalyzer(lang='ru')

In [17]:
morph.parse('ghf')[0].tag.POS

In [18]:
neg_lemma = []
pos_lemma = []
for review in neg_prep:
    rev_lemma = []
    for word in review:
        features = morph.parse(word)[0]
        if not(features.tag.POS): #на этом моменте я избавляюсь от пунктуации и мусора
            continue
        rev_lemma.append(features.normal_form)
    neg_lemma.append(rev_lemma)

for review in pos_prep:
    rev_lemma = []
    for word in review:
        features = morph.parse(word)[0]
        if not(features.tag.POS):
            continue
        rev_lemma.append(features.normal_form)
    pos_lemma.append(rev_lemma)

# 3. Словари

In [19]:
from collections import Counter

Создаю частотные словари для лемм из положительных и отрицательных отзывов

In [20]:
neg_count = Counter([word for review in neg_lemma for word in review])
pos_count = Counter([word for review in pos_lemma for word in review])

Отдельно беру сами слова, отдельно значения для них, чтобы реализовать фильтр по значенниям

In [21]:
neg_words = np.array([i for i in neg_count.keys()])
neg_vals = np.array([i for i in neg_count.values()])
pos_words = np.array([i for i in pos_count.keys()])
pos_vals = np.array([i for i in pos_count.values()])

Оставляю те слова, что встретились больше 4 раз

In [22]:
neg_set = set(neg_words[neg_vals > 4])
pos_set = set(pos_words[pos_vals > 4])

Создаю массивы со словами только из положительных отзывов и только из отрицательных

In [23]:
unique_neg = neg_set.difference(pos_set)
unique_pos = pos_set.difference(neg_set)

# 4. Определитель тональности

## 4.1. Функция

In [24]:
def tone_pred(texts, pos_set, neg_set):
    y_pred = []
    for text in texts:
        val = 0
        for token in casual_tokenize(text, preserve_case=False): #токенизирую отзыв, тональность которого угадывается
            features = morph.parse(token)[0] 
            if not(features.tag.POS): #не учитываю пунктуацию и мусор
                continue
            lemma = features.normal_form #лемматиизирую
            if lemma in pos_set: #если слово находится в положительном списке, то к оценке тональности прибавляю 1
                val += 1
            if lemma in neg_set: #если слово находится в отрицательном списке, то от оценки тональности отнимаю 1
                val -= 1
        if val > 0: #смотрю на знак оценки тональности и по нему делаю предсказание
            y_pred.append(1)
        if val < 0:
            y_pred.append(-1)
        if val == 0:
            y_pred.append(0)
    return(y_pred)

## 4.2. Accuracy

In [25]:
from sklearn.metrics import accuracy_score

In [26]:
accuracy_score(y, tone_pred(test, unique_pos, unique_neg))

0.7

# 5. Улучшения

1. Можно избавиться от стоп-слов
2. Можно ранжировать характерные слова и чем чаще встречается то или иное слово, тем больший вклад в ответ оно вносит

Реализую и то, и другое

In [27]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vito\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Загружаю список стоп-слов

In [28]:
stop_words = stopwords.words('russian')

Сортирую по убыванию слова из отрицательных и положительных отзывов для их дальнейшего ранжирования

In [29]:
neg_sorted = sorted(neg_count, key=neg_count.get, reverse=True)
pos_sorted = sorted(pos_count, key=pos_count.get, reverse=True)

Удаляю слова из списка стоп-слов

In [30]:
for word in neg_sorted:
    if word in stop_words:
        neg_sorted.remove(word)
for word in pos_sorted:
    if word in stop_words:
        pos_sorted.remove(word)

Присваиваю ранги словам так, чтобы у слов с одинаковой встречаемостью ранг был одинаковым

In [31]:
neg_ranged = {}
pos_ranged = {}

In [32]:
for i in range(len(neg_sorted)):
    if i == 0:
        neg_ranged[neg_sorted[i]] = 1 #у первого слова ранг 1
    else:
        if neg_count[neg_sorted[i]] == neg_count[neg_sorted[i-1]]:
            #если слово встретилось столько же раз, сколько и слово до, то ранг одинаковый
            neg_ranged[neg_sorted[i]] = neg_ranged[neg_sorted[i-1]]
        else:
            #если нет, то есть втретилось меньше, то ранг увеличивается на 1
            neg_ranged[neg_sorted[i]] = neg_ranged[neg_sorted[i-1]] + 1
    if neg_count[neg_sorted[i]] < 5:
        #если появляются слова, которые встретились меньше 5 раз, то останавливаемся
        break

То же повторяем для слов из положительных отзывов:

In [33]:
for i in range(len(pos_sorted)):
    if i == 0:
        pos_ranged[pos_sorted[i]] = 1
    else:
        if pos_count[pos_sorted[i]] == pos_count[pos_sorted[i-1]]:
            pos_ranged[pos_sorted[i]] = pos_ranged[pos_sorted[i-1]]
        else:
            pos_ranged[pos_sorted[i]] = pos_ranged[pos_sorted[i-1]] + 1
    if pos_count[pos_sorted[i]] < 5:
        break

Теперь приписываем словам веса: вес = 0.9 ^ ранг (я просто взял удобную убывающую функцию)

In [34]:
neg_weights = {}
pos_weights = {}

In [35]:
for word, rank in neg_ranged.items():
    neg_weights[word] = 0.9**rank

In [36]:
for word, rank in pos_ranged.items():
    pos_weights[word] = 0.9**rank

Переписываю функцию

In [37]:
def tone_pred(texts, pos_set, neg_set):
    y_pred = []
    for text in texts:
        val = 0
        for token in casual_tokenize(text, preserve_case=False):
            features = morph.parse(token)[0]
            if not(features.tag.POS):
                continue
            lemma = features.normal_form
            if lemma in pos_set.keys():
                val += pos_set[lemma] #для слов из положительного списка -- прибавляю его вес
            if lemma in neg_set.keys():
                val -= neg_set[lemma] #для слова из отрицательного -- отнимаю
        if val > 0:
            y_pred.append(1)
        if val < 0:
            y_pred.append(-1)
        if val == 0:
            y_pred.append(0)
    return(y_pred)

In [38]:
accuracy_score(y, tone_pred(test, pos_weights, neg_weights))

0.4

Что ж, результат стал хуже, но я пытался